# Testing the navigation model

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from model import InstructNav

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cfg = {
    'obs_dim': 20,
    'pred_dim': 16,
    'lstm_hidden_dim': 10,
    'lstm_num_layers': 4,
    'env_out_dim': 256,
    'lang_out_dim': 256,
    'lin_mlp_shape': [256, 32],
    'ang_mlp_shape': [256, 32],
    'num_xttn_heads': 2
}
model = InstructNav(cfg=cfg)
model

C:\Users\dev-sys\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
C:\Users\dev-sys\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
C:\Users\dev-sys\AppData\Roaming\Python\Python311\site-packages\torch\nn

InstructNav(
  (pc_encoder): PointNetEncoder(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fstn): STNkd(
      (conv1): Conv1d(64, 64, kernel_

Number of trainable parameters

In [4]:
def model_summary(model):
    '''
    Function to display the number of trainable, non-trainable and total parameters in a model
    '''
    from tabulate import tabulate
    def count_parameters(module):
        trainable_params = sum(p.numel() for p in module.parameters() if p.requires_grad)
        non_trainable_params = sum(p.numel() for p in module.parameters() if not p.requires_grad)
        return trainable_params, non_trainable_params, trainable_params + non_trainable_params

    def count_parameters_by_submodule(model):
        submodule_params = []
        total_trainable = 0
        total_non_trainable = 0
        for name, submodule in model.named_children():
            trainable_params, non_trainable_params, total_params = count_parameters(submodule)
            total_trainable += trainable_params
            total_non_trainable += non_trainable_params
            submodule_params.append([name, trainable_params, non_trainable_params, total_params])
        total = ["Total", total_trainable, total_non_trainable, total_trainable + total_non_trainable]
        return submodule_params, total
    
    headers = ["Submodule", "Trainable Params", "Non-trainable Params", "Total Params"]
    submodule_params, total = count_parameters_by_submodule(model)
    print(tabulate(submodule_params + [total], headers=headers, tablefmt="grid"))

In [5]:
model_summary(model)

+----------------+--------------------+------------------------+----------------+
| Submodule      |   Trainable Params |   Non-trainable Params |   Total Params |
+================+====================+========================+================+
| pc_encoder     |            2803529 |                      0 |        2803529 |
+----------------+--------------------+------------------------+----------------+
| vision_encoder |                  0 |              309823488 |      309823488 |
+----------------+--------------------+------------------------+----------------+
| lang_encoder   |                  0 |             1674121216 |     1674121216 |
+----------------+--------------------+------------------------+----------------+
| lstm           |             126000 |                      0 |         126000 |
+----------------+--------------------+------------------------+----------------+
| env_fc         |             786688 |                      0 |         786688 |
+---------------

In [8]:
model.parameters

<bound method Module.parameters of InstructNav(
  (pc_encoder): PointNetEncoder(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fstn): STNkd(
  

In [43]:
import torch
a = torch.rand(5, 2, 3, 5)
a[1].shape

torch.Size([2, 3, 5])

In [26]:
import matplotlib.pyplot as plt
import torch
import numpy as np

cmap = plt.get_cmap('Blues')
img = torch.randint(0, 255, (256, 256), dtype=torch.uint8).float().numpy()
img[1, 1] = np.nan
img = cmap(img)
img[1, 1]

array([0., 0., 0., 0.])

In [27]:
a = [1, 2, 3]
b = None
a + b

TypeError: can only concatenate list (not "NoneType") to list